In [12]:
import hashlib
import time
import json


class Block:
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        self.index = index
        self.transactions = transactions  # list of dict
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.merkle_root = self.compute_merkle_root()
        self.hash = self.compute_hash()

    def compute_merkle_root(self):
        def hash_pair(a, b):
            return hashlib.sha256((a + b).encode()).hexdigest()

        tx_hashes = [hashlib.sha256(json.dumps(tx, sort_keys=True).encode()).hexdigest()
                     for tx in self.transactions]

        while len(tx_hashes) > 1:
            if len(tx_hashes) % 2 == 1:  # make even
                tx_hashes.append(tx_hashes[-1])
            tx_hashes = [hash_pair(tx_hashes[i], tx_hashes[i + 1])
                         for i in range(0, len(tx_hashes), 2)]

        return tx_hashes[0] if tx_hashes else ''

    def compute_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "transactions": self.transactions,
            "timestamp": self.timestamp,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce,
            "merkle_root": self.merkle_root
        }, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

In [23]:
import time
import hashlib
from multiprocessing import Process, Value, cpu_count
from typing import List

def mine_worker(start_nonce, step, block: Block, difficulty: int, result_nonce: Value, found_flag: Value):
    print("Miner start")
    time.sleep(3)
    # prefix = '0' * difficulty
    # nonce = start_nonce
    # 
    # while not found_flag.value:
    #     block.nonce = nonce
    #     current_hash = block.compute_hash()
    #     if current_hash.startswith(prefix):
    #         with result_nonce.get_lock(), found_flag.get_lock():
    #             if not found_flag.value:
    #                 print(f"Found result nonce: {nonce}")
    #                 result_nonce.value = nonce
    #                 found_flag.value = 1
    #         return
    #     nonce += step


def mine_block_multiprocessing(block: Block, difficulty: int, processes: int = 4):
    result_nonce = Value('i', 2)
    found_flag = Value('i', 0)

    # workers = []
    # for i in range(4):
    #     p = Process(target=mine_worker, args=(i*1000, 1, block, difficulty, result_nonce, found_flag))
    #     p.start()
    #     workers.append(p)
    # 
    # for p in workers:
    #     p.join()
    p = Process(target=mine_worker, args=(1, 10000, block, 3, result_nonce, found_flag))
    p.start()
    p.join()

    # Gán lại nonce và hash chính xác vào block
    block.nonce = result_nonce.value
    block.hash = block.compute_hash()

    print(f" Nonce found: {result_nonce.value}")
    print(block.hash)

0
0


In [22]:
new_block = Block(
    index=1,
    transactions=[{"from": "Alice", "to": "Bob", "amount": 10}],
    timestamp=time.time(),
    previous_hash=123213
)

result_nonce = Value('i', 0)
found_flag = Value('i', 0)

# mine_worker(1,10000,new_block,3,result_nonce,found_flag)
p = Process(target=mine_worker, args=(1,10000,new_block,3,result_nonce,found_flag))
p.start()
p.join()

print(result_nonce.value)
print(found_flag.value)

# mine_block_multiprocessing(new_block, 4)

0
0


In [15]:
# new_block.nonce = result_nonce.value
# print(new_block.compute_hash())

print(new_block.nonce)

0


In [16]:
class Blockchain:
    def __init__(self, difficulty=2):
        self.chain = []
        self.difficulty = difficulty
        self.pending_transactions = []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        self.chain.append(genesis_block)

    def get_last_block(self):
        return self.chain[-1]

    def add_transaction(self, transaction):
        self.pending_transactions.append(transaction)

    def mine_pending_transactions(self):
        if not self.pending_transactions:
            print("No transactions to mine.")
            return

        new_block = Block(
            index=len(self.chain),
            transactions=self.pending_transactions,
            timestamp=time.time(),
            previous_hash=self.get_last_block().hash
        )

        print("Mining block...")

        # while not new_block.hash.startswith('0' * self.difficulty):
        #     new_block.nonce += 1
        #     new_block.hash = new_block.compute_hash()

        # mine_block_multiprocessing(new_block, difficulty=self.difficulty, processes=cpu_count())

        print(f"Block mined: {new_block.hash}")
        self.chain.append(new_block)
        self.pending_transactions = []

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            curr = self.chain[i]
            prev = self.chain[i - 1]

            if curr.hash != curr.compute_hash():
                return False
            if curr.previous_hash != prev.hash:
                return False
            if not curr.hash.startswith('0' * self.difficulty):
                return False
        return True


In [17]:
bc = Blockchain(difficulty=4)




In [18]:
bc.add_transaction({"from": "Alice", "to": "Bob", "amount": 10})
bc.add_transaction({"from": "Bob", "to": "Charlie", "amount": 5})

bc.mine_pending_transactions()

for block in bc.chain:
    print(f"\nBlock {block.index}")
    print(f"Hash: {block.hash}")
    print(f"Prev: {block.previous_hash}")
    print(f"Transactions: {block.transactions}")

print(bc.is_chain_valid())

Mining block...
Block mined: 132b9a367dd2b6534c39fa9f8ffd8aa9bee971f5ac3b8bfea1deb4ad5ab62a7b

Block 0
Hash: 4b6a5be9f8948d1948b45004bbd7be1bee5704e557ab80e9a1406f2423944a3b
Prev: 0
Transactions: []

Block 1
Hash: 132b9a367dd2b6534c39fa9f8ffd8aa9bee971f5ac3b8bfea1deb4ad5ab62a7b
Prev: 4b6a5be9f8948d1948b45004bbd7be1bee5704e557ab80e9a1406f2423944a3b
Transactions: [{'from': 'Alice', 'to': 'Bob', 'amount': 10}, {'from': 'Bob', 'to': 'Charlie', 'amount': 5}]
False


In [19]:
def worker(name):
    print(f"Process {name} bắt đầu")
    time.sleep(2)
    print(f"Process {name} kết thúc")


In [20]:
p1 = Process(target=worker, args=("A",))
p2 = Process(target=worker, args=("B",))

p1.start()
p2.start()

p1.join()
p2.join()

print("Hoàn tất tất cả tiến trình.")

Hoàn tất tất cả tiến trình.
